<a href="https://colab.research.google.com/github/DeokhyeonLee/pythonstudy/blob/master/%E1%84%83%E1%85%A2%E1%84%87%E1%85%A5%E1%86%B8%E1%84%8B%E1%85%AF%E1%86%AB%E1%84%89%E1%85%A1%E1%84%80%E1%85%A5%E1%86%AB%E1%84%80%E1%85%A5%E1%86%B7%E1%84%89%E1%85%A2%E1%86%A8%E1%84%8F%E1%85%B3%E1%84%85%E1%85%A9%E1%86%AF%E1%84%85%E1%85%B5%E1%86%BC2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import pandas as pd
import numpy as np
import webbrowser
import re
from pandas import Series, DataFrame
from bs4 import BeautifulSoup
from urllib.request import urlopen
#from selenium import webdriver
#from selenium.webdriver.support.ui import Select
import time

## 엑셀 파일 불러오기

In [0]:
#데이터 가져오기
csv_data = pd.read_csv('court.csv', header=None)
court = []
sagun = []
name = []
for i in range(1, len(csv_data)):
    x = csv_data[0][i]
    court.append(x)
    y = csv_data[1][i]
    sagun.append(y)
    z = csv_data[2][i]
    name.append(z)

#리스트 초기화 
a0 = [] #사건번호 - Index0
a1 = [] #접수일_Index3
a2 = [] #개시결정일_Index6
a3 = [] #채권이의마감일_Index7
a4 = [] #변제계획인가일_Index8
a5 = [] #면책결정일_Index9
a6 = [] #절차폐지결정일_Index10
cw = []

## 드라이버 및 사이트 지정

[]


In [0]:
drv = webdriver.Chrome('/Users/deokhyeon/chromedriver_mac64/chromedriver')
#newdrv = webdriver.Chrome('/Users/deokhyeon/chromedriver_mac64/chromedriver')
url = 'http://safind.scourt.go.kr/sf/mysafind.jsp?sch_sa_gbn=&sch_bub_nm=&sa_year=&sa_gubun=&sa_serial=&x=&y=&saveCookie='
#soup = BeautifulSoup(requests.get(url).text, "html.parser")
options = webdriver.ChromeOptions()
cnt = 0
drv.implicitly_wait(10)
drv.get(url)
#체크박스 체크
drv.find_element_by_xpath("//*[@id='inputsano_ch']").click()
drv.find_element_by_xpath("//*[@id='BubSort']").click()    
drv.find_element_by_xpath("//*[@id='SagbnSort']").click()
drv.find_element_by_xpath("//*[@id='saveCookieYn']").click()


## 엑셀에 저장된 데이터 자동입력하는 모듈

In [0]:
#관할법원
drv.find_element_by_xpath("//select[@name='sch_bub_nm']/option[text()='"+court[cnt]+"']").click()

#사건번호
input_sano = drv.find_element_by_xpath("//*[@name='input_sano']")
input_sano.send_keys(sagun[cnt])

#채무자명 입력
ds_nm = drv.find_element_by_xpath("//*[@id='ds_nm']")
ds_nm.send_keys(name[cnt])

#자동문자 방지 입력
input_answer = drv.find_element_by_xpath("//*[@id='answer']")

capcha = input()
input_answer.send_keys(capcha)

#검색버튼 클릭
drv.find_element_by_xpath("//*[@class='redBtn']").click()
cnt += 1

#사건일반내용의 기본내용 파싱
html = drv.page_source
soup = BeautifulSoup(html, 'html.parser')
time.sleep(1)

table = soup.find('table', {'class':'tableVer'})
listt = table.find_all('th') #항목이름
listt2 = table.find_all('td') #항목내용

#인덱스별 접근이 아니라 전체를 파싱한 다음에 필요한 항목만 불러올 수 있게끔 바꿔보자
a0.append(listt2[0].text.strip())
a1.append(listt2[3].text.strip())
a2.append(listt2[6].text.strip())
a3.append(listt2[7].text.strip())
a4.append(listt2[8].text.strip())
a5.append(listt2[9].text.strip())
a6.append(listt2[10].text.strip())


#사건일반내용의 당사자 파싱
#x = soup.select("#subTab1 > table:nth-child(11) > tbody > tr > td.tit")
#for i in range (len(x)):
#    y = x[i].get_text()
#    if '한국주택금융공사' in y:
#        cw.append(y)
    
#사건진행내용 클릭
element = drv.find_element_by_class_name("subTab2")
drv.execute_script("goMove('0');", element)
#time.sleep(1)

#나의 사건 검색하기
#drv.find_element_by_xpath(".//a[@href='/sf/mysafind.jsp?theme=scourt']").click()


#test 나의 사건검색하기 누르기
#drv.find_element_by_xpath("//*[@href='http://safind.scourt.go.kr/sf/mysafind.jsp?theme=scourt']").click()

714097


## 데이프레임 만든 후 csv파일로 저장

In [0]:
#리스트 다 만든 다음에 실행하기
df_sa_num = pd.DataFrame(a0, columns = ['사건번호'])
df_jeopjuil = pd.DataFrame(a1, columns=['접수일'])
df_gaesiil = pd.DataFrame(a2, columns = ['개시결정일'])
df_Iuiil = pd.DataFrame(a3, columns = ['채권이의마감일'])
df_ingail = pd.DataFrame(a4, columns = ['변제계획인가일'])
df_myeonil = pd.DataFrame(a5, columns = ['면책결정일'])
df_pyejiil = pd.DataFrame(a6, columns = ['절차폐지결정일'])

In [0]:
df = pd.concat([df_sa_num, df_jeopjuil, df_gaesiil, df_Iuiil, df_ingail, df_myeonil, df_pyejiil], axis=1)
df.to_csv('Hakjageum.csv', index = False, encoding="euc-kr")

In [0]:
df

,사건번호,접수일,개시결정일,채권이의마감일,변제계획인가일,면책결정일,절차폐지결정일
0,2016개회41908,2016.09.19,2017.01.16,2017.02.28,2017.04.04,,
1,2018개회1043001,2018.06.28,2018.07.20,2018.08.20,2018.10.04,,
2,2014개회191060,2014.10.07,2015.01.30,2015.03.19,2015.04.29,,


In [0]:
html = drv.page_source
soup = BeautifulSoup(html, 'html.parser')
#사건 진행내용 파싱
#z = soup.find_all("td", class_="tit").get_text()
#print(z)
z = soup.select("#subTab2 > table.tableHor > tbody > tr > td[style]")
len(z)
z1 = []
z2 = []
z3 = []
for i in range (len(z)):
    a = z[i].text.strip()
    if '한국주택금융공사' in a:
        z1 = (z[i-1]).text().strip()
        z2 = (z[i]).get_text().lstrip()
        z3 = (z[i+1]).get_text()

    #a = z[i].text.strip()
print(z1)

TypeError: 'str' object is not callable

In [0]:
for i in range 
z1 = soup.select("#subTab2 > table.tableHor > tbody > tr:nth-child(1) > td:nth-child(1)")
z2 = soup.select("#subTab2 > table.tableHor > tbody > tr:nth-child(1) > td.tit")
z3 = soup.select("#subTab2 > table.tableHor > tbody > tr:nth-child(1) > td:nth-child(3)")


In [0]:
   for i in range(len(x)):
        z1.append(x[i].text)
        coName.append(listt[cnt].find("td", {"class":"tit"}).text)
        coRating.append(listt[cnt].find_all(text=re.compile("^[A-C]")))
        ratingYear.append(listt[cnt].find_all(text=re.compile("^[2]\d{3}")))
        cnt += 1
        x += 10

In [0]:

#subTab2 > table.tableHor > tbody > tr:nth-child(1) > td:nth-child(1)
#subTab2 > table.tableHor > tbody > tr:nth-child(1) > td.tit
#subTab2 > table.tableHor > tbody > tr:nth-child(1) > td:nth-child(3)

In [0]:
x[1].find_all('td')

[<td style="color: #660000;"> 2017.11.27</td>,
 <td class="tit" style="color: #660000;"> 채무자 강O호 금지명령신청 제출</td>,
 <td style="color: #660000;">  </td>]

In [0]:
file_format = ['son', 'seungha']
file_name = 'My name is seungha'
 
if any(format in file_name for format in file_format):
    print(format)



<built-in function format>


In [0]:
coName = []
coRating = []
ratingYear = []
ratingCo = []
df_ratingCo = pd.DataFrame(ratingCo, columns = ['RatingCo'])
df_coName = pd.DataFrame(coName, columns=['CompanyName'])
df_coRating = pd.DataFrame(coRating, columns = ['Rating'])
df_ratingYear = pd.DataFrame(ratingYear, columns = ['Day'])

df = pd.concat([df_ratingCo, df_coName, df_coRating, df_ratingYear], axis=1)
df

,RatingCo,CompanyName,Rating,Day


In [0]:
for i in range (len(x)):
    y = x[i].get_text()
    if '한국주택금융공사' in y:
        print(y)
        
print(type(y))

[<th>사건번호</th>, <th>사건명</th>, <th>재판부</th>, <th>접수일</th>, <th>채무자</th>, <th>회생위원</th>, <th>개시결정일</th>, <th>채권이의마감일</th>, <th>변제계획인가일</th>, <th>면책결정일</th>, <th>절차폐지결정일</th>, <th>종국결과</th>, <th colspan="2">변제현황조회</th>, <th colspan="2">송달료, 보관금, 종결에 따른 잔액조회</th>]


In [0]:
print(cw)

['\xa0채권번호4. 한국주택금융공사 ', '\xa0채권번호2. 한국주택금융공사 ', '\xa0채권번호1. 한국주택금융공사 ', '\xa0채권번호2. 한국주택금융공사 ']


In [0]:
drv.quit()

In [0]:
#사건일반내용 파싱
html = drv.page_source
soup = BeautifulSoup(html, 'html.parser')

#기본내용
table = soup.find('table', {'class':'tableVer'})
listt = table.find_all('th') #항목이름
listt2 = table.find_all('td') #항목내용

#인덱스별 접근이 아니라 전체를 파싱한 다음에 필요한 항목만 불러올 수 있게끔 바꿔보자
a1.append(listt2[3].text)
a2.append(listt2[6].text)
a3.append(listt2[7].text)
a4.append(listt2[8].text)
a5.append(listt2[9].text)
a6.append(listt2[10].text)

#당사자 내용 파싱
soup.select("#subTab1 > table:nth-child(11) > tbody")

for i in range(1,len(listt)):
    x = 1
    a1.append(listt2[x].text)
    coName.append(listt[cnt].find("td", {"class":"tit"}).text)
    coRating.append(listt[cnt].find_all(text=re.compile("^[A-C]")))
    ratingYear.append(listt[cnt].find_all(text=re.compile("^[2]\d{3}")))
    cnt += 1
    x += 10


In [0]:
#사건일반내용의 당사자 파싱
cw = []
html = drv.page_source
bs = BeautifulSoup(html, 'html.parser')
x = bs.select("#subTab1 > table:nth-child(11) > tbody > tr > td.tit")
for i in range (len(x)):
    y = x[i].get_text()
    if '한국주택금융공사' in y:
        cw.append(y)
    
print(cw)

IndexError: list index out of range

In [0]:
table = soup.find('table', {'class':'tableVer'})
listt = table.find_all('th') #항목이름
listt2 = table.find_all('td') #항목내용

print(type(table))

<class 'bs4.element.Tag'>


In [0]:
drv.find_element_by_xpath("//*[@id='subTab1']/table[4]/tbody")

<selenium.webdriver.remote.webelement.WebElement (session="a71b9598e8bf9bdf65471ddc6f4d237a", element="0b4aab18-cb3c-4743-bf64-1eda46d75dfa")>

In [0]:
#subTab1 > div:nth-child(10)
#subTab1 > table:nth-child(11) > tbody
#subTab1 > table:nth-child(11) > tbody

In [0]:
print(a1)

['\xa02014.11.03']


In [0]:
#사건진행내용 클릭
element = drv.find_element_by_class_name("subTab2")
drv.execute_script("goMove('0');", element)

In [0]:
#나의 사건 검색하기
drv.find_element_by_xpath(".//a[@href='/sf/mysafind.jsp?theme=scourt']").click()

In [0]:
drv.quit()

In [0]:
listt = table.find_all('th') #항목이름
listt2 = table.find_all('td') #항목내용

In [0]:
drv.find_element_by_css_selector('tab1 > div.subTab2 > ul > li.btn > div > a:nth-child(2)').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"tab1 > div.subTab2 > ul > li.btn > div > a:nth-child(2)"}
  (Session info: chrome=76.0.3809.132)


## 사건검색내용 텍스트 추출

### 사건일반내용

In [0]:
#리스트 초기화 
a1 = [] #접수일_Index3
a2 = [] #개시결정일_Index6
a3 = [] #채권이의마감일_Index7
a4 = [] #변제계획인가일_Index8
a5 = [] #면책결정일_Index9
a6 = [] #절차폐지결정일_Index10

In [0]:
#공백제거
a1 = [line.strip() for line in a1]
a2 = [line.strip() for line in a2]
a3 = [line.strip() for line in a3]
a4 = [line.strip() for line in a4]
a5 = [line.strip() for line in a5]
a6 = [line.strip() for line in a6]

In [0]:
print(a1)

['2016.09.19', '2016.09.19', '2018.06.28', '2014.10.07', '2017.11.27', '2014.11.03', '2018.02.07', '2014.11.26', '2014.10.23', '2015.04.30']


### 엑셀에 데이터에 입력시키기

### 사건 진행내용 클릭!

In [0]:
html = drv.page_source
soup = BeautifulSoup(html, 'html.parser')

In [0]:
#사건 진행내용은 순서가 매번 바뀌니 인덱스 값이 아니라 반복문 이용해서
#공사가 있는지 없는지를 찾아야겠군

In [0]:
#사건진행내용 클릭
element = drv.find_element_by_class_name("subTab2")
drv.execute_script("goMove('0');", element)

NameError: name 'time' is not defined

In [0]:
print(soup)

<html lang="ko"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no, target-densitydpi=medium-dpi" name="viewport"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>회생파산사건(WHP) 일반내용</title>
<link href="/common/css/reset_safind.css" rel="stylesheet" type="text/css"/>
<link href="/common/css/layout_safind.css" rel="stylesheet" type="text/css"/>
<script language="javascript" src="/common/js/jquery.js" type="text/javascript"></script>
<script language="javascript" src="/common/js/basic.js" type="text/javascript"></script>
<script language="javascript" src="/js/SfCommon.js" type="text/javascript"></script>
<script language="javascript" type="text/Javascript">
//<![CDATA[
  window.document.oncontextmenu = new Function("return false");
//]]>
</script>
</head>
<body style="">
<form action="" method="post" name="print_preview">
<input name="

In [0]:
#사건검색내역 클릭
drv.find_element_by_xpath("//*[@id='resultList']/table/tbody/tr/td[4]/b/a").click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='resultList']/table/tbody/tr/td[4]/b/a"}
  (Session info: chrome=76.0.3809.132)


In [0]:
drv.find_element_by_xpath("//*[@id='tab1']/div[2]/ul/li[3]/div/a[2]")

<selenium.webdriver.remote.webelement.WebElement (session="7dbcefc986e50c8747efbb0365998591", element="a4fe66a7-f451-47ff-9a5d-5f3cf4476e0d")>

In [0]:
sample = drv.find_element_by_css_selector('a')
drv.execute_script("arguments[0].click();", sample)  #자바 명령어 실행

In [0]:
element = drv.find_element_by_xpath("//td[@class='tit']/b")
drv.execute_script("arguments[0].click();", element)

In [0]:
drv.find_element_by_xpath("//*[@href='#']").click()

In [0]:
drv.quit()

In [0]:
for i in range(1, len(csv_data)-1):
    #새로운 탭 열고 이동
    drv.execute_script("window.open('http://safind.scourt.go.kr/sf/mysafind.jsp?sch_sa_gbn=&sch_bub_nm=&sa_year=&sa_gubun=&sa_serial=&x=&y=&saveCookie=');")
    drv.switch_to_window(drv.window_handles[i])
    
        
    # 사람처럼 보이게 하는 옵션들
    #options.add_argument("disable-gpu")   # 가속 사용 x
    #options.add_argument("lang=ko_KR")    # 가짜 플러그인 탑재
    
    drv.find_element_by_xpath("//*[@id='inputsano_ch']").click()
    drv.find_element_by_xpath("//*[@id='BubSort']").click()    
    drv.find_element_by_xpath("//*[@id='SagbnSort']").click()
    drv.find_element_by_xpath("//*[@id='saveCookieYn']").click()
    
    #데이터 입력
    #관할법원
    drv.find_element_by_xpath("//select[@name='sch_bub_nm']/option[text()='"+court[i]+"']").click() #관할법원
    #print(court[i])

    
    #사건번호
    input_sano = drv.find_element_by_xpath("//*[@name='input_sano']")
    input_sano.send_keys(sagun[i])
    
    #채무자명 입력
    ds_nm = drv.find_element_by_xpath("//*[@id='ds_nm']")
    ds_nm.send_keys(name[i])
    
    #자동문자 방지 입력
    input_answer = drv.find_element_by_xpath("//*[@id='answer']")

    capcha = input()
    input_answer.send_keys(capcha)

    #검색버튼 클릭
    drv.find_element_by_xpath("//*[@class='redBtn']").click()

    #갯수증가
    cnt += 1

//anaconda3/envs/deokenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: use driver.switch_to.window instead
  after removing the cwd from sys.path.


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=76.0.3809.132)


In [0]:
drv.quit()

In [0]:
#자동문자 방지 입력
input_answer = drv.find_element_by_xpath("//*[@id='answer']"

9


In [0]:
#for i in range(5):
drv.find_element_by_xpath("//select[@name='sch_bub_nm']/option[text()='"+court[0]+"']").click() #관할법원
    #drv.find_element_by_xpath("//*[@id='inputsano_ch']").click()
    #cnt += 1
    #사건입력
    

In [0]:
drv.find_element_by_xpath("//*[@id='inputsano_ch']").click()
drv.find_element_by_xpath("//*[@id='BubSort']").click()
drv.find_element_by_xpath("//*[@id='SagbnSort']").click()
drv.find_element_by_xpath("//*[@id='saveCookieYn']").click()

In [0]:
#사건번호 입력
input_sano = drv.find_element_by_xpath("//*[@name='input_sano']")
input_sano.send_keys('222')

#drv.find_element_by_xpath("//select[@name='sel_sa_year']/option[text()='2016']").click() #2016을 사건년도 반복문으로 바꾸기
#drv.find_element_by_xpath("//select[@name='sa_gubun']/option[text()='개회']").click() 
#sa_serial = drv.find_element_by_xpath("//*[@id='sa_serial']")
#sa_serial.send_keys('41908') #사건번호

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=76.0.3809.132)


In [0]:
#채무자명 입력
ds_nm = drv.find_element_by_xpath("//*[@id='ds_nm']")
ds_nm.send_keys('강성현')

#체크박스 체크하기

drv.find_element_by_xpath("//*[@id='BubSort']").click()
drv.find_element_by_xpath("//*[@id='SagbnSort']").click()
drv.find_element_by_xpath("//*[@id='saveCookieYn']").click()

In [0]:
drv.find_element_by_xpath("//select[@name='sch_bub_nm']/option[text()='서울회생법원']").click() #관할법원
#drv.find_element_by_xpath("//*[@id='inputsano_ch']").click()
#사건번호로 입력
input_sano = drv.find_element_by_xpath("//*[@name='input_sano']")
input_sano.send_keys('2018개회1043001')

#drv.find_element_by_xpath("//select[@name='sel_sa_year']/option[text()='2016']").click() #2016을 사건년도 반복문으로 바꾸기
#drv.find_element_by_xpath("//select[@name='sa_gubun']/option[text()='개회']").click() 
#sa_serial = drv.find_element_by_xpath("//*[@id='sa_serial']")
#sa_serial.send_keys('41908') #사건번호
#채무자명 입력
ds_nm = drv.find_element_by_xpath("//*[@id='ds_nm']")
ds_nm.send_keys('강소영')

#체크박스 체크하기

#drv.find_element_by_xpath("//*[@id='BubSort']").click()
#drv.find_element_by_xpath("//*[@id='SagbnSort']").click()
#drv.find_element_by_xpath("//*[@id='saveCookieYn']").click()

In [0]:
print(csv_data)

        0              1     2         3        4           5      6   \
0     관할법원           사건번호  채무자명  접수일(신청일)  금지명령송달일  채권자계좌번호신고일  개시결정일   
1   대전지방법원    2016개회41908   강성현       NaN      NaN         NaN    NaN   
2   서울회생법원  2018개회1043001   강소영       NaN      NaN         NaN    NaN   
3   서울회생법원   2014개회191060   강금순       NaN      NaN         NaN    NaN   
4   울산지방법원    2017개회20924   강명호       NaN      NaN         NaN    NaN   
5   대전지방법원    2014개회61649   강미리       NaN      NaN         NaN    NaN   
6   서울회생법원     2018개회9459   강민성       NaN      NaN         NaN    NaN   
7   전주지방법원   2014개회100402   강민정       NaN      NaN         NaN    NaN   
8   수원지방법원   2014개회147238   강사랑       NaN      NaN         NaN    NaN   
9   인천지방법원    2015개회37355   강소연       NaN      NaN         NaN    NaN   
10  수원지방법원    2017개회54054   강아론       NaN      NaN         NaN    NaN   

                 7                8      9        10  
0   채권이의마감일(이의신청기간)  변제계획인가일(이의신청기간)  면책결정일  절차폐지결정일  
1            

In [0]:
print(len(csv_data))
print(len(csv_data.columns))

11
11


In [0]:
csv_data[2]

0     채무자명
1      강성현
2      강소영
3      강금순
4      강명호
5      강미리
6      강민성
7      강민정
8      강사랑
9      강소연
10     강아론
Name: 2, dtype: object

In [0]:
x = []
for i in range(1, len(csv_data)):
    y = csv_data[0][i]
    x.append(y)


In [0]:
print(x)

['대전지방법원', '서울회생법원', '서울회생법원', '울산지방법원', '대전지방법원', '서울회생법원', '전주지방법원', '수원지방법원', '인천지방법원', '수원지방법원']


In [0]:
for i in range(0, 2):
    for j in range(1, len(csv_data.columns)):
        print(csv_data)

        0              1     2         3        4           5      6   \
0     관할법원           사건번호  채무자명  접수일(신청일)  금지명령송달일  채권자계좌번호신고일  개시결정일   
1   대전지방법원    2016개회41908   강성현       NaN      NaN         NaN    NaN   
2   서울회생법원  2018개회1043001   강소영       NaN      NaN         NaN    NaN   
3   서울회생법원   2014개회191060   강금순       NaN      NaN         NaN    NaN   
4   울산지방법원    2017개회20924   강명호       NaN      NaN         NaN    NaN   
5   대전지방법원    2014개회61649   강미리       NaN      NaN         NaN    NaN   
6   서울회생법원     2018개회9459   강민성       NaN      NaN         NaN    NaN   
7   전주지방법원   2014개회100402   강민정       NaN      NaN         NaN    NaN   
8   수원지방법원   2014개회147238   강사랑       NaN      NaN         NaN    NaN   
9   인천지방법원    2015개회37355   강소연       NaN      NaN         NaN    NaN   
10  수원지방법원    2017개회54054   강아론       NaN      NaN         NaN    NaN   

                 7                8      9        10  
0   채권이의마감일(이의신청기간)  변제계획인가일(이의신청기간)  면책결정일  절차폐지결정일  
1            

In [0]:
drv.find_element_by_xpath("//*[@href='/sf/mysafind.jsp?theme=scourt']").click()

In [0]:
drv.findElement(By.cssSelector("body")).sendKeys(Keys.CONTROL +"t");

AttributeError: 'WebDriver' object has no attribute 'findElement'

In [0]:
select = Select(drv.find_element_by_id('sch_bub_nm'))


In [0]:
# select by visible text
select.select_by_visible_text('대법원')

In [0]:
#text_main = drv.page_source
#bs_text_main = BeautifulSoup(text_main, "html.parser")

a = soup.find_all('div', {'class' : 'breakDiv'})

print(a)


[<div class="breakDiv">사건번호</div>, <div class="breakDiv">
<label for="sch_bub_nm">법원목록</label>
<select class="graySelect" id="sch_bub_nm" name="sch_bub_nm" onchange="onChangeBub();" title="법원 목록선택">
<option value="">법원선택</option>
</select>
</div>, <div class="breakDiv" id="div_sa_year">
<label for="sel_sa_year">년도 목록선택 </label>
<select class="graySelect" id="sel_sa_year" name="sel_sa_year" title="년도 목록선택">
<option value="2019">2019</option>
<option value="2018">2018</option>
<option value="2017">2017</option>
<option value="2016">2016</option>
<option value="2015">2015</option>
<option value="2014">2014</option>
<option value="2013">2013</option>
<option value="2012">2012</option>
<option value="2011">2011</option>
<option value="2010">2010</option>
<option value="2009">2009</option>
<option value="2008">2008</option>
<option value="2007">2007</option>
<option value="2006">2006</option>
<option value="2005">2005</option>
<option value="2004">2004</option>
<option value="2003">2003</opt

In [0]:
table = soup.find('table', {'class' : 'bbs_list'})
listt = table.find_all("tr")
listt2 = table.find_all('td')

In [0]:
text_main = drv.page_source
bs_text_main = BeautifulSoup(text_main, "html.parser")

In [0]:
a = soup.find_all('div', {'class' : 'subTabContents'})

In [0]:
print(a)

[<div class="content" id="content">
<h2><div class="conTitle">나의 사건검색</div></h2>
<div id="contants">
<iframe frameborder="0" height="1100" marginheight="0" marginwidth="0" scrolling="auto" src="http://safind.scourt.go.kr/sf/mysafind.jsp?sch_sa_gbn=&amp;sch_bub_nm=&amp;sa_year=&amp;sa_gubun=&amp;sa_serial=&amp;x=&amp;y=&amp;saveCookie=" title="나의 사건검색" width="100%"></iframe>
<noframes title="나의사건검색(새창)">
<a href="http://safind.scourt.go.kr/sf/mysafind.jsp?sch_sa_gbn=&amp;sch_bub_nm=&amp;sa_year=&amp;sa_gubun=&amp;sa_serial=&amp;x=&amp;y=&amp;saveCookie=" target="_blank" title="나의사건검색(새창)">프레임이 보이지 않을경우 이곳을 클릭해주세요</a>
</noframes>
</div>
</div>]
